In [35]:
import heapq


class MaxHeap:
    def __init__(self, arr = None):
        self.data = []
        if arr is not None and len(arr) > 0:
            t = type(arr[0]) 

            # Our MaxHeap support ints, floats, or tuples of ints/floats.
            # Note this check is not actually exhaustive.
            self.is_tuple = False
            if t == tuple:
                self.is_tuple = True 
                t = type(arr[0][0])
            if not (t == int or t == float):
                raise TypeError('Must be numeric data.')
            
            self.data = [self._negative(a) for a in arr]
            heapq.heapify(self.data)
    
    def heappush(self, item):
        heapq.heappush(self.data, self._negative(item))

    def _negative(self, item):
        if type(item) == tuple:
            return tuple(-a for a in item)
        else:
            return -item
    
    def heappop(self):
        item = heapq.heappop(self.data)
        return self._negative(item)
    
    def get_largest(self):
        largest = self._negative(self.data[0])
        return largest
    
    def empty(self):
        return len(self.data) == 0
    

In [29]:
# This block of code is all copied from suffix-array-construction. Any bugs, fix there!

# First, we implement the 'doubling' algorithm to compute the suffix array in O(n log n) time.
# Step 1: Comput N_1[i].

import collections
import math
import random

def compute_n1(S):
    letter_count = collections.defaultdict(int)
    sorted_letters = sorted(set([letter for letter in S]))

    for letter in S:
        letter_count[letter] += 1

    letters_before = dict()
    running_total = 0
    for l in sorted_letters:
        letters_before[l] = running_total
        running_total += letter_count[l]

    n = len(S)
    N_1 = [0 for i in range(n)]
    for i in range(n):
        l = S[i]
        N_1[i] = letters_before[l]
    
    return N_1


# Implement ternary quicksort. Will be used in Step 2 below.
def ternary_quicksort(A):    
    n = len(A)
    if n <= 4:
        return sorted(A)
    
    i = random.randint(0, n - 1)
    split_element = A[i]
    A_less = []
    A_eq = []
    A_greater = []
    for a in A:
        if a < split_element:
            A_less.append(a)
        elif a == split_element:
            A_eq.append(a)
        else:
            A_greater.append(a)
    A_sorted = ternary_quicksort(A_less) + A_eq + ternary_quicksort(A_greater)
    return A_sorted

# Step 2: From N_l, compute N_2l.
def compute_n2l(N_l, l):
    n = len(N_l)
    def N_i_plus_l(i, l):
        if i + l >= n:
            return n
        else:
            return N_l[i + l]

    N_l_pairs = []
    index_to_pair = dict()
    for i in range(n):
        pair = (N_l[i], N_i_plus_l(i, l))
        index_to_pair[i] = pair
        N_l_pairs.append(pair)

    N_l_pairs_sorted = ternary_quicksort(N_l_pairs)


    pairs_before = dict()
    prev = None
    count = 0
    for i in range(n):
        pair = N_l_pairs_sorted[i]
        if pair == prev:
            count += 1
        else:
            pairs_before[pair] = count
            count += 1
            prev = pair

    N_2l = [0 for i in range(n)]
    for i in range(n):
        pair = index_to_pair[i]
        N_2l[i] = pairs_before[pair]
    return N_2l



# We assume that S has already been padded with '$' at the end.
def compute_inverse_suffix_array(S):
    n = len(S)
    l = 1
    N_l = compute_n1(S)
    k = math.ceil(math.log2(n))
    l_max = 2 ** k
    while l <= l_max:
        N_2l = compute_n2l(N_l, l)
        l = l * 2
        N_l = N_2l
    return N_l


def compute_suffix_array(S):
    n = len(S)
    SA_inverse = compute_inverse_suffix_array(S)
    SA = [-1] * n
    for i in range(n):
        SA[SA_inverse[i]] = i
    return SA, SA_inverse


def compute_longest_common_prefixes(SA, SA_inverse, T):
    n = len(T)
    # LCP[i] = longest common prefix between SA[i] and SA[i - 1]
    LCP = [-1] * n
    l = 0
    
    # We know SA_inverse[n - 1] = 0 (the last character is the special '$')
    # We don't want to compute LCP[0], so we go to i = n - 2
    for i in range(n - 1):
        k = SA_inverse[i]  # What order is the prefix Ti
        j = SA[k - 1]      # Prefix Tj comes immediately before Ti in lexicographic order
        while T[i + l] == T[j + l]:
            l = l + 1
        LCP[k] = l
        if l > 0:
            l = l - 1
    return LCP

    

In [25]:
t = 'bananas'
T = t + '$' 
SA, SAi = compute_suffix_array(T)
LCP = compute_longest_common_prefixes(SA, SAi, T)
print("T: ", T)
print("SA: ", SA)
print("SAi: ", SAi)
print("LCP: ", LCP)

k  4 j  5
k  1 j  7
k  5 j  0
k  2 j  1
k  6 j  2
k  3 j  3
k  7 j  4
T:  bananas$
SA:  [7, 1, 3, 5, 0, 2, 4, 6]
SAi:  [4, 1, 5, 2, 6, 3, 7, 0]
LCP:  [-1, 0, 3, 1, 0, 0, 2, 0]


In [91]:
def maxValue(t):
    T = t + '$'
    SA, SA_inverse = compute_suffix_array(T)
    # LCP[i] is the amount of overlap between i'th suffix and (i - 1)'th suffix.
    LCP = compute_longest_common_prefixes(SA, SA_inverse, T)
    
    n = len(T)  # Note, n - 1 = len(t)
    max_heap = MaxHeap()
    max_score = n - 1  # The length of string t itself is a lower bound.
    forward_scores = [0 for i in range(n)]
    backward_scores = [0 for i in range(n)]
    for i in range(1, n):
        lcp = LCP[i]
        max_heap.heappush((lcp, i))
        largest, _ = max_heap.get_largest()
        while lcp < largest:
            prior_lcp, prior_i = max_heap.heappop()
            # We add 1 because a longest common prefix happens twice by definition.
            score = (i - prior_i) * prior_lcp
            forward_scores[prior_i] = score
            largest, _ = max_heap.get_largest()

    while not max_heap.empty():
        lcp, j = max_heap.heappop()
        score = (n - j) * lcp
        forward_scores[j] = score

    LCP.reverse()
    for i in range(1, n):
        lcp = LCP[i]
        max_heap.heappush((lcp, i))
        largest, _ = max_heap.get_largest()
        while lcp < largest:
            prior_lcp, prior_i = max_heap.heappop()
            score = (i - prior_i) * prior_lcp
            backward_scores[n - 1 - prior_i] = score
            largest, _ = max_heap.get_largest()

    while not max_heap.empty():
        lcp, j = max_heap.heappop()
        score = (n - j) * lcp
        backward_scores[n - 1 - j] = score

    scores = [max(forward_scores[i], 0) + max(backward_scores[i], 0) for i in range(n)]
    
    return max(scores)






In [59]:
t = '...'

In [83]:
T = t + '$'
SA, SAi = compute_suffix_array(T)
LCP = compute_longest_common_prefixes(SA, SAi, T)
print(LCP)

KeyboardInterrupt: 

100000
807254


In [94]:
t2 = 'abcabcddd'
maxValue(t2)

i, lcp:  1 0
i, lcp:  2 3
i, lcp:  3 0
i, lcp:  4 2
i, lcp:  5 0
i, lcp:  6 1
i, lcp:  7 0
i, lcp:  8 1
i, lcp:  9 2
i, lcp:  1 1
i, lcp:  2 0
i, lcp:  3 1
i, lcp:  4 0
i, lcp:  5 2
i, lcp:  6 0
i, lcp:  7 3
i, lcp:  8 0
i, lcp:  9 -1
[0, 0, 3, 0, 2, 0, 1, 0, 2, 2]
[-1, 0, 3, 0, 2, 0, 1, 0, 1, 0]
[0, 0, 6, 0, 4, 0, 2, 0, 3, 2]


6